In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os

PROJECT_NAME = "Plant-Disease-Detection-ViT"
PROJECT_PATH = f"/content/drive/MyDrive/{PROJECT_NAME}"

folders = [
    PROJECT_PATH,
    f"{PROJECT_PATH}/data",
    f"{PROJECT_PATH}/src",
    f"{PROJECT_PATH}/models",
    f"{PROJECT_PATH}/results",
    f"{PROJECT_PATH}/docs",
]

for folder in folders:
    os.makedirs(folder, exist_ok=True)

print("✅ Project structure created!")
print(f"📁 Location: {PROJECT_PATH}")

In [ ]:
!pip install torch torchvision torchaudio -q
!pip install transformers timm -q
!pip install scikit-learn matplotlib seaborn -q
!pip install Pillow tqdm pandas -q
!pip install kaggle -q

import torch
print(f"✅ Packages installed!")
print(f"🖥️ CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
from google.colab import files
import os

print("📥 STEP 1: Upload your kaggle.json file")
print("   Get it from: https://www.kaggle.com/settings → API → Create New Token")
print("")

uploaded = files.upload()

os.makedirs('/root/.kaggle', exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

print("\n✅ Kaggle API configured!")

In [ ]:
PROJECT_PATH = "/content/drive/MyDrive/Plant-Disease-Detection-ViT"
DATA_PATH = f"{PROJECT_PATH}/data"

print("🌿 Downloading PlantVillage Dataset from Kaggle...")
print("   Dataset: https://www.kaggle.com/datasets/abdallahalidev/plantvillage-dataset")
print("")

!kaggle datasets download -d abdallahalidev/plantvillage-dataset -p {DATA_PATH}

print("\n📦 Extracting dataset...")
!unzip -q {DATA_PATH}/plantvillage-dataset.zip -d {DATA_PATH}

!rm {DATA_PATH}/plantvillage-dataset.zip

print("\n✅ Dataset downloaded and extracted!")

for item in os.listdir(DATA_PATH):
    print(f"   📁 {item}")

In [ ]:
import os

PROJECT_PATH = "/content/drive/MyDrive/Plant-Disease-Detection-ViT"
DATA_PATH = f"{PROJECT_PATH}/data"

DATA_DIR = None

for root, dirs, files in os.walk(DATA_PATH):
    if len(dirs) > 10:
        sample_dir = dirs[0] if dirs else ""
        if '___' in sample_dir or 'healthy' in sample_dir.lower() or 'leaf' in sample_dir.lower():
            DATA_DIR = root
            break

if DATA_DIR is None:
    for root, dirs, files in os.walk(DATA_PATH):
        if len(dirs) > 20:
            DATA_DIR = root
            break

if DATA_DIR:
    print(f"✅ Dataset found: {DATA_DIR}")
    classes = sorted(os.listdir(DATA_DIR))
    print(f"📊 Total classes: {len(classes)}")
    print(f"\n📁 Classes:")
    for i, cls in enumerate(classes[:10]):
        cls_path = os.path.join(DATA_DIR, cls)
        if os.path.isdir(cls_path):
            count = len(os.listdir(cls_path))
            print(f"   {i+1}. {cls}: {count} images")
    if len(classes) > 10:
        print(f"   ... and {len(classes)-10} more classes")
else:
    print("❌ Dataset not found!")

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import random
import os

if DATA_DIR:
    classes = sorted([d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))])
    sample_classes = random.sample(classes, min(12, len(classes)))
    
    fig, axes = plt.subplots(3, 4, figsize=(16, 12))
    axes = axes.flatten()
    
    for idx, cls in enumerate(sample_classes):
        cls_path = os.path.join(DATA_DIR, cls)
        images = [f for f in os.listdir(cls_path) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
        img_name = random.choice(images)
        img_path = os.path.join(cls_path, img_name)
        
        img = Image.open(img_path)
        axes[idx].imshow(img)
        title = cls.replace('___', '\n').replace('_', ' ')[:30]
        axes[idx].set_title(title, fontsize=8)
        axes[idx].axis('off')
    
    plt.suptitle('PlantVillage Dataset - Sample Images', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig(f"{PROJECT_PATH}/results/sample_images.png", dpi=150)
    plt.show()
    
    print(f"✅ Sample images saved!")

In [ ]:
dataset_code = '''
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split

class PlantDiseaseDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

def get_transforms(image_size=224, is_training=True):
    if is_training:
        return transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.3),
            transforms.RandomRotation(20),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    else:
        return transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

def load_dataset(data_dir, test_size=0.2, val_size=0.1, random_state=42):
    image_paths = []
    labels = []
    class_names = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])
    class_to_idx = {cls: idx for idx, cls in enumerate(class_names)}
    
    print(f"Loading dataset from: {data_dir}")
    print(f"Found {len(class_names)} classes")
    
    for cls in class_names:
        cls_path = os.path.join(data_dir, cls)
        for img_name in os.listdir(cls_path):
            if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_paths.append(os.path.join(cls_path, img_name))
                labels.append(class_to_idx[cls])
    
    print(f"Total images: {len(image_paths)}")
    
    train_val_paths, test_paths, train_val_labels, test_labels = train_test_split(
        image_paths, labels, test_size=test_size, random_state=random_state, stratify=labels
    )
    
    val_ratio = val_size / (1 - test_size)
    train_paths, val_paths, train_labels, val_labels = train_test_split(
        train_val_paths, train_val_labels, test_size=val_ratio, random_state=random_state, stratify=train_val_labels
    )
    
    print(f"Train: {len(train_paths)}, Val: {len(val_paths)}, Test: {len(test_paths)}")
    
    return train_paths, val_paths, test_paths, train_labels, val_labels, test_labels, class_names

def create_dataloaders(data_dir, batch_size=32, image_size=224, test_size=0.2, val_size=0.1, num_workers=2):
    train_paths, val_paths, test_paths, train_labels, val_labels, test_labels, class_names = load_dataset(data_dir, test_size, val_size)
    
    train_transform = get_transforms(image_size, is_training=True)
    val_transform = get_transforms(image_size, is_training=False)
    
    train_dataset = PlantDiseaseDataset(train_paths, train_labels, train_transform)
    val_dataset = PlantDiseaseDataset(val_paths, val_labels, val_transform)
    test_dataset = PlantDiseaseDataset(test_paths, test_labels, val_transform)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
    
    return train_loader, val_loader, test_loader, class_names
'''

with open(f"{PROJECT_PATH}/src/dataset.py", "w") as f:
    f.write(dataset_code)

print("✅ src/dataset.py created!")

In [ ]:
model_code = '''
import torch
import torch.nn as nn
import timm

class PlantDiseaseViT(nn.Module):
    def __init__(self, num_classes, model_name='vit_base_patch16_224', pretrained=True, dropout=0.3):
        super(PlantDiseaseViT, self).__init__()
        
        self.vit = timm.create_model(model_name, pretrained=pretrained, num_classes=0)
        self.feature_dim = self.vit.num_features
        
        self.classifier = nn.Sequential(
            nn.LayerNorm(self.feature_dim),
            nn.Dropout(dropout),
            nn.Linear(self.feature_dim, 512),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes)
        )
        
        print(f"Model: {model_name}")
        print(f"Feature dimension: {self.feature_dim}")
        print(f"Number of classes: {num_classes}")
        print(f"Total parameters: {sum(p.numel() for p in self.parameters()):,}")
    
    def forward(self, x):
        features = self.vit(x)
        logits = self.classifier(features)
        return logits

class PlantDiseaseViTSmall(nn.Module):
    def __init__(self, num_classes, pretrained=True, dropout=0.3):
        super(PlantDiseaseViTSmall, self).__init__()
        
        self.vit = timm.create_model('vit_small_patch16_224', pretrained=pretrained, num_classes=0)
        self.feature_dim = self.vit.num_features
        
        self.classifier = nn.Sequential(
            nn.LayerNorm(self.feature_dim),
            nn.Dropout(dropout),
            nn.Linear(self.feature_dim, 256),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(256, num_classes)
        )
        
        print(f"Model: ViT-Small")
        print(f"Total parameters: {sum(p.numel() for p in self.parameters()):,}")
    
    def forward(self, x):
        features = self.vit(x)
        logits = self.classifier(features)
        return logits

def get_model(num_classes, model_type='base', pretrained=True, dropout=0.3):
    if model_type == 'base':
        return PlantDiseaseViT(num_classes, pretrained=pretrained, dropout=dropout)
    elif model_type == 'small':
        return PlantDiseaseViTSmall(num_classes, pretrained=pretrained, dropout=dropout)
    else:
        raise ValueError(f"Unknown model type: {model_type}")
'''

with open(f"{PROJECT_PATH}/src/model.py", "w") as f:
    f.write(model_code)

print("✅ src/model.py created!")

In [ ]:
train_code = '''
import torch
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm
import time
import matplotlib.pyplot as plt

class Trainer:
    def __init__(self, model, train_loader, val_loader, learning_rate=1e-4, weight_decay=0.01, device='cuda', save_path='models/best_model.pth'):
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        self.save_path = save_path
        
        self.criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
        self.optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
        self.scaler = GradScaler()
        
        self.history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'lr': []}
        self.best_val_acc = 0.0
        
    def train_epoch(self):
        self.model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        pbar = tqdm(self.train_loader, desc='Training')
        
        for images, labels in pbar:
            images = images.to(self.device)
            labels = labels.to(self.device)
            
            self.optimizer.zero_grad()
            
            with autocast():
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
            
            self.scaler.scale(loss).backward()
            self.scaler.step(self.optimizer)
            self.scaler.update()
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            pbar.set_postfix({'loss': f'{loss.item():.4f}', 'acc': f'{100.*correct/total:.2f}%'})
        
        return running_loss / len(self.train_loader), 100. * correct / total
    
    def validate(self):
        self.model.eval()
        running_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for images, labels in tqdm(self.val_loader, desc='Validating'):
                images = images.to(self.device)
                labels = labels.to(self.device)
                
                with autocast():
                    outputs = self.model(images)
                    loss = self.criterion(outputs, labels)
                
                running_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
        
        return running_loss / len(self.val_loader), 100. * correct / total
    
    def train(self, epochs, patience=10):
        print(f"{'='*60}")
        print(f"STARTING TRAINING")
        print(f"{'='*60}")
        print(f"Device: {self.device}")
        print(f"Epochs: {epochs}")
        print(f"{'='*60}")
        
        scheduler = optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=epochs, eta_min=1e-6)
        patience_counter = 0
        start_time = time.time()
        
        for epoch in range(epochs):
            print(f"\\nEpoch {epoch+1}/{epochs}")
            
            train_loss, train_acc = self.train_epoch()
            val_loss, val_acc = self.validate()
            
            scheduler.step()
            current_lr = scheduler.get_last_lr()[0]
            
            self.history['train_loss'].append(train_loss)
            self.history['train_acc'].append(train_acc)
            self.history['val_loss'].append(val_loss)
            self.history['val_acc'].append(val_acc)
            self.history['lr'].append(current_lr)
            
            print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
            print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
            
            if val_acc > self.best_val_acc:
                self.best_val_acc = val_acc
                torch.save({'epoch': epoch, 'model_state_dict': self.model.state_dict(), 'optimizer_state_dict': self.optimizer.state_dict(), 'val_acc': val_acc, 'history': self.history}, self.save_path)
                print(f"✅ New best model saved! Val Acc: {val_acc:.2f}%")
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f"\\n⚠️ Early stopping triggered")
                    break
        
        print(f"\\n{'='*60}")
        print(f"TRAINING COMPLETE!")
        print(f"Total time: {(time.time()-start_time)/60:.2f} minutes")
        print(f"Best validation accuracy: {self.best_val_acc:.2f}%")
        print(f"{'='*60}")
        
        return self.history
    
    def plot_history(self, save_path=None):
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))
        
        axes[0].plot(self.history['train_loss'], label='Train', color='blue')
        axes[0].plot(self.history['val_loss'], label='Val', color='red')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Loss')
        axes[0].set_title('Loss')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        axes[1].plot(self.history['train_acc'], label='Train', color='blue')
        axes[1].plot(self.history['val_acc'], label='Val', color='red')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('Accuracy (%)')
        axes[1].set_title('Accuracy')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        axes[2].plot(self.history['lr'], color='green')
        axes[2].set_xlabel('Epoch')
        axes[2].set_ylabel('Learning Rate')
        axes[2].set_title('Learning Rate')
        axes[2].grid(True, alpha=0.3)
        
        plt.tight_layout()
        if save_path:
            plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.show()
'''

with open(f"{PROJECT_PATH}/src/train.py", "w") as f:
    f.write(train_code)

print("✅ src/train.py created!")

In [ ]:
eval_code = '''
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from torch.cuda.amp import autocast
from tqdm import tqdm

class Evaluator:
    def __init__(self, model, test_loader, class_names, device='cuda'):
        self.model = model.to(device)
        self.test_loader = test_loader
        self.class_names = class_names
        self.device = device
        self.predictions = None
        self.true_labels = None
        self.probabilities = None
    
    def evaluate(self):
        self.model.eval()
        all_predictions = []
        all_labels = []
        all_probs = []
        
        with torch.no_grad():
            for images, labels in tqdm(self.test_loader, desc='Evaluating'):
                images = images.to(self.device)
                with autocast():
                    outputs = self.model(images)
                    probs = torch.softmax(outputs, dim=1)
                    _, predicted = outputs.max(1)
                all_predictions.extend(predicted.cpu().numpy())
                all_labels.extend(labels.numpy())
                all_probs.extend(probs.cpu().numpy())
        
        self.predictions = np.array(all_predictions)
        self.true_labels = np.array(all_labels)
        self.probabilities = np.array(all_probs)
        
        return self.get_metrics()
    
    def get_metrics(self):
        return {
            'accuracy': accuracy_score(self.true_labels, self.predictions) * 100,
            'precision': precision_score(self.true_labels, self.predictions, average='weighted', zero_division=0) * 100,
            'recall': recall_score(self.true_labels, self.predictions, average='weighted', zero_division=0) * 100,
            'f1': f1_score(self.true_labels, self.predictions, average='weighted', zero_division=0) * 100,
        }
    
    def print_report(self):
        metrics = self.get_metrics()
        print(f"\\n{'='*60}")
        print(f"EVALUATION RESULTS")
        print(f"{'='*60}")
        print(f"Accuracy:  {metrics['accuracy']:.2f}%")
        print(f"Precision: {metrics['precision']:.2f}%")
        print(f"Recall:    {metrics['recall']:.2f}%")
        print(f"F1-Score:  {metrics['f1']:.2f}%")
        print(f"{'='*60}")
        print(f"\\nClassification Report:")
        print(classification_report(self.true_labels, self.predictions, target_names=self.class_names, zero_division=0))
        return metrics
    
    def plot_confusion_matrix(self, save_path=None, figsize=(20, 16)):
        cm = confusion_matrix(self.true_labels, self.predictions)
        plt.figure(figsize=figsize)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=self.class_names, yticklabels=self.class_names)
        plt.title('Confusion Matrix', fontsize=16, fontweight='bold')
        plt.xlabel('Predicted', fontsize=12)
        plt.ylabel('Actual', fontsize=12)
        plt.xticks(rotation=45, ha='right', fontsize=8)
        plt.yticks(rotation=0, fontsize=8)
        plt.tight_layout()
        if save_path:
            plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.show()
'''

with open(f"{PROJECT_PATH}/src/evaluate.py", "w") as f:
    f.write(eval_code)

with open(f"{PROJECT_PATH}/src/__init__.py", "w") as f:
    f.write("from .dataset import *\nfrom .model import *\nfrom .train import *\nfrom .evaluate import *\n")

print("✅ src/evaluate.py created!")
print("✅ src/__init__.py created!")

In [ ]:
import sys
import torch
import os

PROJECT_PATH = "/content/drive/MyDrive/Plant-Disease-Detection-ViT"
sys.path.insert(0, PROJECT_PATH)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🖥️ Device: {device}")

from src.dataset import create_dataloaders
from src.model import get_model
from src.train import Trainer

torch.manual_seed(42)

IMAGE_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 30
LEARNING_RATE = 1e-4
MODEL_TYPE = 'small'

DATA_DIR = None
for root, dirs, files in os.walk(f"{PROJECT_PATH}/data"):
    if len(dirs) > 10:
        DATA_DIR = root
        break

if DATA_DIR:
    print(f"✅ Dataset: {DATA_DIR}")
    
    print("\n📦 Creating data loaders...")
    train_loader, val_loader, test_loader, class_names = create_dataloaders(
        data_dir=DATA_DIR,
        batch_size=BATCH_SIZE,
        image_size=IMAGE_SIZE,
        test_size=0.2,
        val_size=0.1,
        num_workers=2
    )
    
    NUM_CLASSES = len(class_names)
    print(f"📊 Classes: {NUM_CLASSES}")
    
    print("\n🧠 Creating model...")
    model = get_model(num_classes=NUM_CLASSES, model_type=MODEL_TYPE, pretrained=True, dropout=0.3)
    
    MODEL_SAVE_PATH = f"{PROJECT_PATH}/models/best_model.pth"
    os.makedirs(f"{PROJECT_PATH}/models", exist_ok=True)
    
    trainer = Trainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        learning_rate=LEARNING_RATE,
        weight_decay=0.01,
        device=device,
        save_path=MODEL_SAVE_PATH
    )
    
    print("\n🚀 Starting training...")
    history = trainer.train(epochs=EPOCHS, patience=10)
    
    os.makedirs(f"{PROJECT_PATH}/results", exist_ok=True)
    trainer.plot_history(save_path=f"{PROJECT_PATH}/results/training_history.png")
    
    print("\n✅ Training complete!")
else:
    print("❌ Dataset not found!")

In [ ]:
from src.evaluate import Evaluator
import torch

print("📊 Loading best model...")
checkpoint = torch.load(MODEL_SAVE_PATH)
model.load_state_dict(checkpoint['model_state_dict'])
print(f"Loaded from epoch {checkpoint['epoch']+1} with {checkpoint['val_acc']:.2f}% accuracy")

evaluator = Evaluator(model=model, test_loader=test_loader, class_names=class_names, device=device)

print("\n🔍 Evaluating...")
metrics = evaluator.evaluate()
final_metrics = evaluator.print_report()

evaluator.plot_confusion_matrix(save_path=f"{PROJECT_PATH}/results/confusion_matrix.png", figsize=(24, 20))

ACCURACY = final_metrics['accuracy']
PRECISION = final_metrics['precision']
RECALL = final_metrics['recall']
F1_SCORE = final_metrics['f1']

print(f"\n{'='*60}")
print(f"🎯 FINAL RESULTS")
print(f"{'='*60}")
print(f"Accuracy:  {ACCURACY:.2f}%")
print(f"Precision: {PRECISION:.2f}%")
print(f"Recall:    {RECALL:.2f}%")
print(f"F1-Score:  {F1_SCORE:.2f}%")
print(f"{'='*60}")

if ACCURACY >= 75:
    print("\n✅ SUCCESS! Above 75% requirement!")
else:
    print("\n⚠️ Below 75%, need more training")

In [ ]:
readme = f'''# 🌿 Plant Disease Detection using Vision Transformer

Automated plant disease detection using Vision Transformer (ViT) for agricultural applications.

## 👥 Team
- **Muhammad Shan Danish** - SU92-MSAIW-F25-033
- **Aiman Batool** - SU92-MSAIW-F25-018

## 📊 Results

| Metric | Value |
|--------|-------|
| Accuracy | {ACCURACY:.2f}% |
| Precision | {PRECISION:.2f}% |
| Recall | {RECALL:.2f}% |
| F1-Score | {F1_SCORE:.2f}% |

## 🎯 SDG Alignment
- **SDG 2: Zero Hunger** - Helping farmers detect crop diseases early

## 🧠 Model Architecture
- Vision Transformer (ViT-Small)
- Pre-trained on ImageNet
- Fine-tuned for plant disease classification
- {NUM_CLASSES} disease classes

## 📁 Dataset
- PlantVillage Dataset
- {NUM_CLASSES} classes of plant diseases
- Source: https://www.kaggle.com/datasets/abdallahalidev/plantvillage-dataset

## 🚀 Usage

```bash
git clone https://github.com/Shandanish/Plant-Disease-Detection-ViT.git
pip install -r requirements.txt
python train.py